In [2]:
from os import chdir
chdir('/Users/lananhnguyen/Desktop/thesis/thesis_code')
import main.packages.unchain_chain as chain
import main.packages.mine_generic as mine_g
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.metrics import mean_squared_error
from dieboldmariano import dm_test

In [587]:
model = 'lasso'
h = 3

In [588]:
def take_y_reals():
    cats = ["food", "energy", "neig", "services"]
    
    concatenated_df = pd.DataFrame()

    for cat in cats:
        cat_df = mine_g.load_excel(f"data/hicp_cat_raw/prc_hicp_{cat}.xlsx", name = f"hicp_{cat}", subset=True, verbose=0)
        #cat_df = cat_df[(cat_df.index > mine_g.train_test_split_date)]
        concatenated_df = pd.concat([concatenated_df, cat_df], axis=1)
    return concatenated_df

def extract_forecast_model_h(forecast_all_cat_df, model, h):
    """
    in 4 df, extract columns with model's name and horizon
    """

    pattern = rf"^{model}.*h_{h}"
    matching_cols = [col for col in forecast_all_cat_df.columns if re.search(pattern, col)]
    df_specific_model_h = forecast_all_cat_df.loc[:, matching_cols]
    return df_specific_model_h

import os

def concatenate_csv_files(folder_path):
    """
    Concatenates CSV files in a folder column-wise into a single DataFrame.

    Args:
    - folder_path (str): Path to the folder containing CSV files.

    Returns:
    - concatenated_df (DataFrame): Concatenated DataFrame.
    """

    csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

    concatenated_df = pd.DataFrame()

    # Iterate through each CSV file
    for file in csv_files:
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)
        
        # Add each file's data as new columns in the concatenated DataFrame
        concatenated_df = pd.concat([concatenated_df, df], axis=1)
        

    return concatenated_df


In [ ]:

y_real = take_y_reals()

# Example usage:
folder_path = "data/forecast_results"
forecast_yoy_all_cat = concatenate_csv_files(folder_path)



In [591]:
new_df = extract_forecast_model_h(forecast_yoy_all_cat, model, h)

date_range = pd.date_range(start=mine_g.train_test_split_date + pd.DateOffset(months=1), end=mine_g.max_X_date, freq='M')
new_df.set_index(date_range, inplace=True)

In [592]:
food_chained = chain.transform_back_chained(new_df.iloc[:, 0], y_real.iloc[:, 0])
food_unchain = chain.unchain_series(food_chained, y_real.iloc[:, 0])

In [593]:
energy_chained = chain.transform_back_chained(new_df.iloc[:, 1], y_real.iloc[:, 1])
energy_unchain = chain.unchain_series(energy_chained, y_real.iloc[:, 1])

In [594]:
neig_chained = chain.transform_back_chained(new_df.iloc[:, 2], y_real.iloc[:, 2])
neig_unchained = chain.unchain_series(neig_chained, y_real.iloc[:, 2])

In [595]:
services_chained = chain.transform_back_chained(new_df.iloc[:, 3], y_real.iloc[:, 3])
services_unchained = chain.unchain_series(services_chained, y_real.iloc[:, 3])

In [596]:
weight_f = pd.read_excel('data/hicp_cat_raw/prc_hicp_weight_food.xlsx', sheet_name='Sheet 1', skiprows=7)

weight_f = weight_f.iloc[1, :].to_frame().reset_index()
weight_f.columns = weight_f.iloc[0]
weight_f = weight_f[1:]
weight_f.rename(columns={"TIME": "date", "Germany": 'weight_f'}, inplace=True)

/Users/lananhnguyen/anaconda3/envs/thesis/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [597]:
weight_e = pd.read_excel('data/hicp_cat_raw/prc_hicp_weight_energy.xlsx', sheet_name='Sheet 1', skiprows=7)

weight_e = weight_e.iloc[1, :].to_frame().reset_index()
weight_e.columns = weight_e.iloc[0]
weight_e = weight_e[1:]
weight_e.rename(columns={"TIME": "date", "Germany": 'weight_e'}, inplace=True)
weight_e

/Users/lananhnguyen/anaconda3/envs/thesis/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,date,weight_e
1,2015,117.76
2,2016,103.55
3,2017,104.74
4,2018,103.74
5,2019,117.28
6,2020,108.7
7,2021,107.65
8,2022,120.65
9,2023,97.04
10,2024,104.27


In [598]:
weight_n = pd.read_excel('data/hicp_cat_raw/prc_hicp_weight_neig.xlsx', sheet_name='Sheet 1', skiprows=7)

weight_n = weight_n.iloc[1, :].to_frame().reset_index()
weight_n.columns = weight_n.iloc[0]
weight_n = weight_n[1:]
weight_n.rename(columns={"TIME": "date", "Germany": 'weight_n'}, inplace=True)
weight_n

/Users/lananhnguyen/anaconda3/envs/thesis/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,date,weight_n
1,2015,266.94
2,2016,269.89
3,2017,267.71
4,2018,267.04
5,2019,266.91
6,2020,265.55
7,2021,275.14
8,2022,266.84
9,2023,282.75
10,2024,279.86


In [599]:
weight_s = pd.read_excel('data/hicp_cat_raw/prc_hicp_weight_services.xlsx', sheet_name='Sheet 1', skiprows=7)

weight_s = weight_s.iloc[1, :].to_frame().reset_index()
weight_s.columns = weight_s.iloc[0]
weight_s = weight_s[1:]
weight_s.rename(columns={"TIME": "date", "Germany": 'weight_s'}, inplace=True)


/Users/lananhnguyen/anaconda3/envs/thesis/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [600]:
weights = pd.concat([weight_f, weight_e, weight_n, weight_s], axis=1)
weights = weights.loc[:, ~weights.columns.duplicated()]
weights['date'] = weights['date'].astype(int)
for col in weights.columns[1:]:
    weights[col] = weights[col].astype(float)

weights.loc[:, 'total'] = weights.iloc[:, 1:5].sum(axis=1)
weights


,date,weight_f,weight_e,weight_n,weight_s,total
1,2015,161.41,117.76,266.94,453.89,1000.0
2,2016,162.03,103.55,269.89,464.53,1000.0
3,2017,165.43,104.74,267.71,462.12,1000.0
4,2018,168.13,103.74,267.04,461.09,1000.0
5,2019,153.56,117.28,266.91,462.25,1000.0
6,2020,155.48,108.70,265.55,470.27,1000.0
7,2021,174.04,107.65,275.14,443.17,1000.0
8,2022,171.53,120.65,266.84,440.98,1000.0
9,2023,166.62,97.04,282.75,453.59,1000.0
10,2024,162.03,104.27,279.86,453.84,1000.0


In [601]:
weights

,date,weight_f,weight_e,weight_n,weight_s,total
1,2015,161.41,117.76,266.94,453.89,1000.0
2,2016,162.03,103.55,269.89,464.53,1000.0
3,2017,165.43,104.74,267.71,462.12,1000.0
4,2018,168.13,103.74,267.04,461.09,1000.0
5,2019,153.56,117.28,266.91,462.25,1000.0
6,2020,155.48,108.70,265.55,470.27,1000.0
7,2021,174.04,107.65,275.14,443.17,1000.0
8,2022,171.53,120.65,266.84,440.98,1000.0
9,2023,166.62,97.04,282.75,453.59,1000.0
10,2024,162.03,104.27,279.86,453.84,1000.0


In [602]:

food_unchain.name = 'food_unchain'
food_unchain = food_unchain.to_frame()
food_unchain.loc[:, 'year'] = food_unchain.index.year
food_unchain
merged_df = pd.merge(food_unchain, weights, how='inner', left_on='year', right_on='date')
mul_food = merged_df.loc[:, 'food_unchain'].mul(merged_df.loc[:, 'weight_f'])

In [603]:
energy_unchain.name = 'e_unchain'
e_unchain = energy_unchain.to_frame()
e_unchain.loc[:, 'year'] = e_unchain.index.year

merged_df = pd.merge(e_unchain, weights, how='inner', left_on='year', right_on='date')
mul_e = merged_df.loc[:, 'e_unchain'].mul(merged_df.loc[:, 'weight_e'])

In [604]:
neig_unchained.name = 'n_unchain'
n_unchain = neig_unchained.to_frame()
n_unchain.loc[:, 'year'] = n_unchain.index.year

merged_df = pd.merge(n_unchain, weights, how='inner', left_on='year', right_on='date')
mul_n = merged_df.loc[:, 'n_unchain'].mul(merged_df.loc[:, 'weight_n'])

In [605]:
services_unchained.name = 's_unchain'
s_unchain = services_unchained.to_frame()
s_unchain.loc[:, 'year'] = s_unchain.index.year

merged_df = pd.merge(s_unchain, weights, how='inner', left_on='year', right_on='date')
mul_s = merged_df.loc[:, 's_unchain'].mul(merged_df.loc[:, 'weight_s'])

In [606]:
mul_sum = pd.concat([mul_food, mul_e, mul_n, mul_s], axis=1)
mul_sum.loc[:, 'hicp_unchained'] = mul_sum.sum(axis=1)/1000

date_range = pd.date_range(start=mine_g.train_test_split_date + pd.DateOffset(months=1), end=mine_g.max_X_date, freq='M')
mul_sum.set_index(date_range, inplace=True)

In [608]:
def transform_to_chained(y_unchain, y_real):
    y_real = y_real[y_real.index <= "2022-12-31"]
    dec_mask = y_real.index.month == 12
    dec_data = y_real.where(dec_mask, other=np.nan)
    dec_data.ffill(inplace=True)
    dec_data = dec_data[dec_data.index > "2015-12-31"]
    # chain data back:
    y_chained = y_unchain * dec_data
    return y_chained


y_real_hicp = mine_g.load_excel('data/hicp_all.xlsx', 'hicp_all', verbose=0)
hicp_chained_final = transform_to_chained(mul_sum.loc[:, 'hicp_unchained'], y_real_hicp.iloc[:, 0])

y_real_hicp.loc[:, 'last_y'] = y_real_hicp.iloc[:, 0].shift(12)
y_real_hicp = y_real_hicp[(y_real_hicp.index > mine_g.train_test_split_date) & (y_real_hicp.index <= mine_g.max_X_date)]

hicp_all_yoy = (hicp_chained_final/y_real_hicp.loc[:, 'last_y']-1)*100
hicp_all_yoy.name = f'{model}_h_{h}'

In [609]:
forecast = pd.read_csv('data/headline_forecast/head_infl_forecast.csv')
new_forecast = pd.concat([forecast, hicp_all_yoy.reset_index(drop=True)], axis=1)
new_forecast

,ar_110_h_1,ar_110_h_2,ar_110_h_3,ar_ols_h_1,ar_ols_h_2,ar_ols_h_3,prc_h_1,prc_h_2,prc_h_3,xgb_h_1,...,xgb_h_3,ols_h_3,ols_h_1,ols_h_2,ridge_h_1,ridge_h_2,ridge_h_3,lasso_h_1,lasso_h_2,lasso_h_3
0,0.201236,0.329779,1.160298,0.305073,0.400090,1.253522,0.477723,0.879217,1.155731,0.720385,...,0.809066,1.758673,1.435635,1.912272,0.812713,0.705458,0.576346,0.883870,0.769113,0.839818
1,0.377993,0.201232,0.333175,0.493679,0.399921,0.486686,0.187900,0.325666,0.682909,0.703246,...,0.439849,1.893299,1.219878,1.454731,0.594620,0.710290,0.642899,0.594509,0.738501,0.865850
2,-0.025772,0.382279,0.201233,0.029098,0.572149,0.486532,-0.262701,0.057059,0.181945,0.345818,...,0.511637,1.455432,1.075196,1.232296,0.062272,0.328006,0.521062,0.070648,0.340993,0.768718
3,0.070035,-0.036898,0.381662,0.212929,0.147909,0.643806,-0.134229,-0.315474,0.018045,0.273920,...,0.273084,1.371610,1.429649,1.217043,0.023588,0.032405,0.232308,0.080892,0.140819,0.520427
4,-0.238173,0.074585,-0.035253,-0.152066,0.315777,0.256404,-0.189562,-0.133276,-0.265374,-0.057299,...,0.278855,1.221701,1.168133,1.404260,-0.232143,-0.240215,-0.238921,-0.340952,-0.190594,0.033996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,8.453284,8.295415,8.659386,7.862867,7.111033,6.988278,8.789301,8.881570,9.070468,6.766554,...,6.174983,7.238727,10.940054,8.786489,9.273274,9.755021,10.180451,9.057146,9.274098,9.353103
80,8.796783,8.455887,8.295984,8.188482,7.301452,6.614902,9.531649,9.304273,9.162557,6.921153,...,5.730577,8.027608,12.065348,10.086081,9.381456,9.764964,10.317768,9.340945,9.363216,9.498422
81,10.712508,8.800719,8.455609,10.074116,7.598793,6.788786,11.255497,10.205313,9.737996,7.790400,...,6.546032,9.379500,13.629764,11.300606,12.189238,11.515665,11.569068,11.549914,11.204452,10.695643
82,11.523992,10.729384,8.800305,10.680094,9.320693,7.060308,11.863813,11.666736,10.465141,8.762138,...,7.488421,10.287110,14.489972,12.539876,12.341713,12.754858,12.426976,11.528633,12.265356,11.908712


In [610]:
new_forecast.to_csv('data/headline_forecast/head_infl_forecast.csv', index=False)

----

In [611]:
real_hicp_yoy = pd.read_csv('data/preprocessed/head_inflation.csv', parse_dates = True, index_col='date')
real_hicp_yoy_test = real_hicp_yoy[(real_hicp_yoy.index > mine_g.train_test_split_date) & (real_hicp_yoy.index <= mine_g.max_X_date)]

new_forecast = pd.concat([new_forecast, real_hicp_yoy_test.reset_index(drop=True)], axis=1)
new_forecast

,ar_110_h_1,ar_110_h_2,ar_110_h_3,ar_ols_h_1,ar_ols_h_2,ar_ols_h_3,prc_h_1,prc_h_2,prc_h_3,xgb_h_1,...,ols_h_3,ols_h_1,ols_h_2,ridge_h_1,ridge_h_2,ridge_h_3,lasso_h_1,lasso_h_2,lasso_h_3,yoy_rate
0,0.201236,0.329779,1.160298,0.305073,0.400090,1.253522,0.477723,0.879217,1.155731,0.720385,...,1.758673,1.435635,1.912272,0.812713,0.705458,0.576346,0.883870,0.769113,0.839818,0.407747
1,0.377993,0.201232,0.333175,0.493679,0.399921,0.486686,0.187900,0.325666,0.682909,0.703246,...,1.893299,1.219878,1.454731,0.594620,0.710290,0.642899,0.594509,0.738501,0.865850,-0.101010
2,-0.025772,0.382279,0.201233,0.029098,0.572149,0.486532,-0.262701,0.057059,0.181945,0.345818,...,1.455432,1.075196,1.232296,0.062272,0.328006,0.521062,0.070648,0.340993,0.768718,0.100301
3,0.070035,-0.036898,0.381662,0.212929,0.147909,0.643806,-0.134229,-0.315474,0.018045,0.273920,...,1.371610,1.429649,1.217043,0.023588,0.032405,0.232308,0.080892,0.140819,0.520427,-0.299401
4,-0.238173,0.074585,-0.035253,-0.152066,0.315777,0.256404,-0.189562,-0.133276,-0.265374,-0.057299,...,1.221701,1.168133,1.404260,-0.232143,-0.240215,-0.238921,-0.340952,-0.190594,0.033996,-0.099404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,8.453284,8.295415,8.659386,7.862867,7.111033,6.988278,8.789301,8.881570,9.070468,6.766554,...,7.238727,10.940054,8.786489,9.273274,9.755021,10.180451,9.057146,9.274098,9.353103,8.834244
80,8.796783,8.455887,8.295984,8.188482,7.301452,6.614902,9.531649,9.304273,9.162557,6.921153,...,8.027608,12.065348,10.086081,9.381456,9.764964,10.317768,9.340945,9.363216,9.498422,10.899183
81,10.712508,8.800719,8.455609,10.074116,7.598793,6.788786,11.255497,10.205313,9.737996,7.790400,...,9.379500,13.629764,11.300606,12.189238,11.515665,11.569068,11.549914,11.204452,10.695643,11.562782
82,11.523992,10.729384,8.800305,10.680094,9.320693,7.060308,11.863813,11.666736,10.465141,8.762138,...,10.287110,14.489972,12.539876,12.341713,12.754858,12.426976,11.528633,12.265356,11.908712,11.261261


# Compare OLS for headline and OLS from individual cat aggregated

In [3]:
forecast = pd.read_csv('data/headline_forecast/head_infl_forecast.csv')

real_hicp_yoy = pd.read_csv('data/preprocessed/head_inflation.csv', parse_dates = True, index_col='date')
real_hicp_yoy_test = real_hicp_yoy[(real_hicp_yoy.index > mine_g.train_test_split_date) & (real_hicp_yoy.index <= mine_g.max_X_date)]


In [631]:
rmse = []

for h in [1, 2, 3]:
    h_cols = []

    for col in forecast.columns:
        if col.endswith(str(h)):
            h_cols.append(col)
    forecast_df_h = forecast[h_cols]
    
    for col in [ col for col in forecast_df_h.columns if "ar_ols" not in col]:
        print(col)
        print(dm_test(real_hicp_yoy_test.iloc[:, 0], forecast_df_h[col], forecast_df_h.loc[:, f'ar_ols_h_{h}'], h = h, harvey_correction=True))


    for col in forecast_df_h.columns:
        rmse_here = np.sqrt(mean_squared_error(forecast_df_h[col], real_hicp_yoy_test))
        rmse.append(rmse_here)


def get_first_part(col_name, separator='_h'):
    return col_name.split(separator)[0]

new_col_names = [get_first_part(col) for col in forecast_df_h.columns]
rmse = pd.DataFrame(rmse)
rmse = pd.DataFrame(rmse.values.reshape(3, int(len(forecast.columns)/3)), columns=new_col_names, index=[f'h_{h}' for h in [1, 2, 3]])


ar_110_h_1
(-1.6769896095002441, 0.0973070531150434)
prc_h_1
(-1.5781254467216208, 0.11834067278353033)
xgb_h_1
(3.4237065320769067, 0.0009619323061758958)
ols_h_1
(3.539041479399328, 0.0006609258505795756)
ridge_h_1
(-1.0653307344634626, 0.289815590386732)
lasso_h_1
(-0.8946303986000829, 0.37357171115027277)
ar_110_h_2
(-1.5284227334846807, 0.13020984156895388)
prc_h_2
(-1.2279405052947059, 0.22294023474817415)
xgb_h_2
(2.3304612418639303, 0.022209175970383455)
ols_h_2
(1.724296670959621, 0.08837689553732869)
ridge_h_2
(-1.4989486791627376, 0.13768095075089137)
lasso_h_2
(-1.505347480195841, 0.13603101380567806)
ar_110_h_3
(-1.4461771591154988, 0.15189327661093388)
prc_h_3
(-1.3639850790723447, 0.17625979361844457)
xgb_h_3
(0.7626246007952472, 0.44784928154163506)
ols_h_3
(0.8765551561984665, 0.3832582640415413)
ridge_h_3
(-1.7327621235060546, 0.08685209010974732)
lasso_h_3
(-1.618374084619781, 0.10937611294124494)


In [633]:
rmse.to_csv("data/report_rmse/total.csv")

In [629]:
rmse_comparative = [round(rmse[col]/rmse[f'ar_ols'], 2) for col in rmse.columns]
rmse_comparative

[h_1    0.90
 h_2    0.82
 h_3    0.78
 dtype: float64,
 h_1    1.0
 h_2    1.0
 h_3    1.0
 Name: ar_ols, dtype: float64,
 h_1    0.85
 h_2    0.79
 h_3    0.74
 dtype: float64,
 h_1    1.66
 h_2    1.32
 h_3    1.04
 dtype: float64,
 h_1    2.19
 h_2    1.39
 h_3    1.16
 dtype: float64,
 h_1    0.9
 h_2    0.7
 h_3    0.6
 dtype: float64,
 h_1    0.93
 h_2    0.72
 h_3    0.65
 dtype: float64]